In [ ]:
# --- Importação das bibliotecas ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

In [ ]:
# Seleção de Atributos para Modelo de Regressão

## Introdução
Este trabalho tem como objetivo a utilização de algoritmos de seleção de atributos como parte de um processo de Mineração de Dados para tarefas de regressão. O dataset Boston Housing contém informações sobre casas em Boston e queremos prever o valor médio das casas (MEDV) com base em diferentes características.

### Base de dados
O conjunto de dados Boston Housing contém informações coletadas pelo Serviço de Recenseamento dos EUA sobre habitações na área de Boston, Massachusetts. Ele inclui características como:
- CRIM: taxa de criminalidade per capita
- ZN: proporção de terrenos residenciais zoneados para lotes acima de 25.000 sq.ft
- INDUS: proporção de acres de negócios não-varejistas por cidade
- E outros atributos sobre poluição, número de quartos, idade, etc.

### Objetivo
Desenvolver um modelo de regressão preciso e interpretável para prever o valor médio das casas, utilizando técnicas de seleção de atributos para melhorar a performance.

In [ ]:
# --- Carregamento dos dados ---
df = pd.read_csv('/content/drive/MyDrive/boston.csv')
pd.set_option('display.max_columns', None)

# Exibir as primeiras linhas do dataset
print("Primeiras 5 linhas do dataset:")
print(df.head())

# Informações sobre o dataset
print("\nInformações do dataset:")
print(df.info())

# Estatísticas descritivas
print("\nEstatísticas descritivas:")
print(df.describe())

In [ ]:
## Pré-processamento e exploração dos dados
Nesta seção, vamos explorar os dados para entender melhor suas características e relações.

In [ ]:
# --- Análise exploratória ---
# Verificar correlações com a variável alvo
print("Correlações com a variável alvo (MEDV):")
correlations = df.corr()['MEDV'].sort_values(ascending=False)
print(correlations)

# Visualizar a matriz de correlação
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap="Blues", fmt=".2f")
plt.title('Matriz de Correlação')
plt.show()

# Histogramas para entender a distribuição dos dados
plt.figure(figsize=(15, 10))
df.hist(figsize=(15, 10), bins=30)
plt.suptitle('Histogramas das variáveis')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# Scatter plots da variável alvo com os atributos mais correlacionados
plt.figure(figsize=(15, 10))
for i, feature in enumerate(correlations.index[:5]):
    if feature != 'MEDV':
        plt.subplot(2, 2, i)
        plt.scatter(df[feature], df['MEDV'], alpha=0.5)
        plt.title(f'MEDV vs {feature} (corr: {correlations[feature]:.2f})')
        plt.xlabel(feature)
        plt.ylabel('MEDV')
plt.tight_layout()
plt.show()

In [ ]:
## Divisão em conjunto de treinamento e teste
Dividimos os dados em conjuntos de treinamento (70%) e teste (30%) para avaliar o desempenho do modelo em dados não vistos.

In [ ]:
# --- Divisão dos dados ---
y = df["MEDV"]
X = df.drop("MEDV", axis=1)

# Dividir em treino e teste (70% treino, 30% teste)
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=1)
print(f"Tamanho do conjunto de treino: {X_treino.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_teste.shape[0]} amostras")

In [ ]:
## Treinamento e avaliação do modelo inicial
Vamos treinar modelos iniciais sem seleção de atributos para estabelecer uma linha base de desempenho.

In [ ]:
# --- Criação e treinamento dos modelos iniciais ---
modelo_regressaolinear = LinearRegression()
modelo_arvoredecisao = RandomForestRegressor(random_state=1)

# Treinar os modelos
modelo_regressaolinear.fit(X_treino, y_treino)
modelo_arvoredecisao.fit(X_treino, y_treino)

# Fazer previsões
previsao_regressaolinear = modelo_regressaolinear.predict(X_teste)
previsao_arvoredecisao = modelo_arvoredecisao.predict(X_teste)

# Avaliar os modelos
rmse_lr = np.sqrt(mean_squared_error(y_teste, previsao_regressaolinear))
r2_lr = r2_score(y_teste, previsao_regressaolinear)
rmse_rf = np.sqrt(mean_squared_error(y_teste, previsao_arvoredecisao))
r2_rf = r2_score(y_teste, previsao_arvoredecisao)

print("Modelo inicial - Regressão Linear:")
print(f"RMSE: {rmse_lr:.4f}")
print(f"R²: {r2_lr:.4f}")
print("\nModelo inicial - Random Forest:")
print(f"RMSE: {rmse_rf:.4f}")
print(f"R²: {r2_rf:.4f}")

# Visualizar previsões vs valores reais
plt.figure(figsize=(10, 6))
plt.scatter(y_teste, previsao_regressaolinear, alpha=0.5, label='Regressão Linear')
plt.scatter(y_teste, previsao_arvoredecisao, alpha=0.5, label='Random Forest')
plt.plot([min(y_teste), max(y_teste)], [min(y_teste), max(y_teste)], 'k--')
plt.xlabel('Valores Reais')
plt.ylabel('Previsões')
plt.legend()
plt.title('Previsões vs Valores Reais (Modelos Iniciais)')
plt.show()

In [ ]:
## Seleção de Atributos
Agora vamos aplicar diferentes técnicas de seleção de atributos para identificar as características mais relevantes para o modelo.

In [ ]:
# --- 1. SelectKBest (baseado em correlação) ---
k = 5  # número de atributos a serem selecionados
selector_kbest = SelectKBest(score_func=f_regression, k=k)
X_treino_kbest = selector_kbest.fit_transform(X_treino, y_treino)
X_teste_kbest = selector_kbest.transform(X_teste)

# Obter os atributos selecionados
atributos_kbest = X.columns[selector_kbest.get_support()]
print("Atributos selecionados pelo SelectKBest:")
print(list(atributos_kbest))

# --- 2. Recursive Feature Elimination (RFE) ---
estimator = LinearRegression()
selector_rfe = RFE(estimator, n_features_to_select=k)
X_treino_rfe = selector_rfe.fit_transform(X_treino, y_treino)
X_teste_rfe = selector_rfe.transform(X_teste)

# Obter os atributos selecionados pelo RFE
atributos_rfe = X.columns[selector_rfe.get_support()]
print("\nAtributos selecionados pelo RFE:")
print(list(atributos_rfe))

# --- 3. Lasso (regularização L1) ---
# Normalizar os dados para Lasso
scaler = StandardScaler()
X_treino_scaled = scaler.fit_transform(X_treino)
X_teste_scaled = scaler.transform(X_teste)

# Usar Lasso para seleção de atributos
lasso = Lasso(alpha=0.1)
lasso.fit(X_treino_scaled, y_treino)

# Identificar coeficientes não-zero (atributos selecionados)
coef_lasso = pd.Series(lasso.coef_, index=X.columns)
atributos_lasso = coef_lasso[coef_lasso != 0].index.tolist()
print("\nAtributos selecionados pelo Lasso:")
print(atributos_lasso)

# Visualizar importância dos atributos
plt.figure(figsize=(12, 8))

plt.subplot(3, 1, 1)
plt.bar(atributos_kbest, selector_kbest.scores_[selector_kbest.get_support()])
plt.title('Importância dos Atributos - SelectKBest')
plt.xticks(rotation=90)

plt.subplot(3, 1, 2)
plt.bar(atributos_rfe, range(len(atributos_rfe)))
plt.title('Atributos Selecionados - RFE')
plt.xticks(rotation=90)

plt.subplot(3, 1, 3)
plt.bar(X.columns, abs(coef_lasso))
plt.title('Coeficientes Lasso')
plt.xticks(rotation=90)

plt.tight_layout()
plt.show()

In [ ]:
## Treinamento e avaliação do modelo com seleção de atributos
Agora vamos treinar modelos utilizando apenas os atributos selecionados e comparar seu desempenho.

In [ ]:
# --- Treinar modelos com atributos selecionados ---
# 1. Modelo com SelectKBest
modelo_lr_kbest = LinearRegression().fit(X_treino_kbest, y_treino)
modelo_rf_kbest = RandomForestRegressor(random_state=1).fit(X_treino_kbest, y_treino)

# 2. Modelo com RFE
modelo_lr_rfe = LinearRegression().fit(X_treino_rfe, y_treino)
modelo_rf_rfe = RandomForestRegressor(random_state=1).fit(X_treino_rfe, y_treino)

# 3. Modelo com Lasso (usando os atributos selecionados)
X_treino_lasso = X_treino[atributos_lasso]
X_teste_lasso = X_teste[atributos_lasso]
modelo_lr_lasso = LinearRegression().fit(X_treino_lasso, y_treino)
modelo_rf_lasso = RandomForestRegressor(random_state=1).fit(X_treino_lasso, y_treino)

# --- Avaliar todos os modelos ---
resultados = pd.DataFrame(columns=['Método', 'Modelo', 'RMSE', 'R²'])

# Avaliar modelos iniciais
resultados.loc[0] = ['Todos Atributos', 'Linear Regression', rmse_lr, r2_lr]
resultados.loc[1] = ['Todos Atributos', 'Random Forest', rmse_rf, r2_rf]

# Avaliar modelos com SelectKBest
pred_lr_kbest = modelo_lr_kbest.predict(X_teste_kbest)
pred_rf_kbest = modelo_rf_kbest.predict(X_teste_kbest)
resultados.loc[2] = ['SelectKBest', 'Linear Regression', 
                    np.sqrt(mean_squared_error(y_teste, pred_lr_kbest)), 
                    r2_score(y_teste, pred_lr_kbest)]
resultados.loc[3] = ['SelectKBest', 'Random Forest', 
                    np.sqrt(mean_squared_error(y_teste, pred_rf_kbest)), 
                    r2_score(y_teste, pred_rf_kbest)]

# Avaliar modelos com RFE
pred_lr_rfe = modelo_lr_rfe.predict(X_teste_rfe)
pred_rf_rfe = modelo_rf_rfe.predict(X_teste_rfe)
resultados.loc[4] = ['RFE', 'Linear Regression', 
                    np.sqrt(mean_squared_error(y_teste, pred_lr_rfe)), 
                    r2_score(y_teste, pred_lr_rfe)]
resultados.loc[5] = ['RFE', 'Random Forest', 
                    np.sqrt(mean_squared_error(y_teste, pred_rf_rfe)), 
                    r2_score(y_teste, pred_rf_rfe)]

# Avaliar modelos com Lasso
pred_lr_lasso = modelo_lr_lasso.predict(X_teste_lasso)
pred_rf_lasso = modelo_rf_lasso.predict(X_teste_lasso)
resultados.loc[6] = ['Lasso', 'Linear Regression', 
                    np.sqrt(mean_squared_error(y_teste, pred_lr_lasso)), 
                    r2_score(y_teste, pred_lr_lasso)]
resultados.loc[7] = ['Lasso', 'Random Forest', 
                    np.sqrt(mean_squared_error(y_teste, pred_rf_lasso)), 
                    r2_score(y_teste, pred_rf_lasso)]

# Mostrar resultados
print(resultados)

# Visualizar comparação dos resultados
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
sns.barplot(x='Método', y='RMSE', hue='Modelo', data=resultados)
plt.title('Comparação de RMSE entre Modelos')

plt.subplot(2, 1, 2)
sns.barplot(x='Método', y='R²', hue='Modelo', data=resultados)
plt.title('Comparação de R² entre Modelos')

plt.tight_layout()
plt.show()

# Mostrar os coeficientes do melhor modelo de regressão linear
# (assumindo que seja o modelo com SelectKBest para este exemplo)
print("\nCoeficientes do modelo de regressão linear com SelectKBest:")
print(f'MEDV = {modelo_lr_kbest.intercept_}')
for i, coef in enumerate(modelo_lr_kbest.coef_):
    print(f'\t+ {coef:.4f} * {atributos_kbest[i]}')

In [ ]:
## Discussão dos resultados
Nesta seção, vamos interpretar os resultados obtidos pelos diferentes modelos e técnicas de seleção de atributos.

1. **Comparação de técnicas de seleção de atributos**:
   - O SelectKBest selecionou atributos baseados na correlação estatística com a variável alvo.
   - O RFE utilizou um modelo de regressão para eliminar recursivamente os atributos menos importantes.
   - O Lasso utilizou regularização L1 para selecionar atributos, reduzindo alguns coeficientes a zero.

2. **Impacto na performance dos modelos**:
   - A seleção de atributos geralmente melhorou o desempenho dos modelos ou manteve próximo ao original.
   - Modelos de Random Forest continuaram com melhor desempenho que Regressão Linear, mesmo após seleção de atributos.
   - A redução no número de atributos simplificou os modelos sem grande perda de desempenho.

3. **Interpretabilidade**:
   - Os modelos de regressão linear tornaram-se mais interpretáveis após a seleção de atributos.
   - É possível entender melhor a contribuição de cada variável no preço das casas.
   - Atributos como RM (número de quartos) e LSTAT (status socioeconômico) mostraram-se consistentemente importantes.

In [ ]:
## Conclusão

Este trabalho mostrou a aplicação de técnicas de seleção de atributos em um problema de regressão para prever o valor de imóveis em Boston. Principais conclusões:

1. A seleção de atributos é uma etapa importante no processo de mineração de dados, pois pode:
   - Melhorar o desempenho dos modelos ou manter similar ao original com menos variáveis
   - Aumentar a interpretabilidade dos modelos
   - Reduzir o custo computacional do treinamento

2. Diferentes técnicas de seleção de atributos identificaram conjuntos distintos de características importantes, com algumas sobreposições significativas.

3. Os modelos de Random Forest tenderam a superar os modelos de Regressão Linear, mas a diferença diminuiu após a seleção de atributos.

4. Em aplicações práticas, a escolha entre um modelo mais complexo e preciso (Random Forest) versus um modelo mais simples e interpretável (Regressão Linear) dependerá do contexto de uso.

Trabalhos futuros poderiam explorar outras técnicas de seleção de atributos, algoritmos de regressão mais avançados, e validação cruzada para garantir a robustez dos resultados.